In [13]:
import cv2
import numpy as np
from skimage import color, data, restoration
from skimage.filters import threshold_otsu

# #########  general functions ###########
def getImg(path):
    try:
        img = cv2.imread(path)
        return img
    except Exception as e:
        message = handleExcept('getImg', e)
        print message
    

def grayImg(img):
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return gray
    except Exception as e:
        message = handleExcept('grayImg', e)
        print message    


def handleExcept(filterName, exception):
    errmsg = str(exception)
    message = '{} : {}'.format(filterName, errmsg) 
    #print message
    return message


def report(filterName):
    message = '{} : succeed'.format(filterName)
    #print message
    return message


def diff_scale(img):
    if len(img.shape) <=2 or img.shape[2] == 1:
        raise RuntimeError('Input img is not a three-chennel array')
    img1 = np.zeros(img.shape[:-1], dtype = img.dtype)
    img1[:,:] = img[:,:,0]*0.114+img[:,:,1]*0.587+img[:,:,2]*0.299

    img2 = np.zeros(img.shape[:-1], dtype = img.dtype)
    img2[:,:] = img[:,:,0]*0.333+img[:,:,1]*0.333+img[:,:,2]*0.333
    return normalize(img2*0.99 - img1)


def normalize(img):
    img_back = img.copy()
    cv2.normalize(img, img_back, 0, 255, cv2.NORM_MINMAX)
    return img_back


# ###################   filters   #####################
def shapeFilter(gray):
    RES = 9
    MESSAGE = ''
    try:
        # blur
        bl = cv2.bilateralFilter(gray,5,50,50)

        # threshold
        ret,output = cv2.threshold(bl,100,255,cv2.THRESH_BINARY)

        # erosion & dilation for clean up the image
        target = cv2.dilate(output, None, 7)
        output = cv2.erode(target, None, 5)

        # find contours
        (cnts, hierarchy) = cv2.findContours(output.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        c = sorted(cnts, key = cv2.contourArea, reverse = True)
        pad_cnt = c[0]

        # covex hull
        hull = cv2.convexHull(pad_cnt)
        hull_area = cv2.contourArea(hull)

        # approx polyDP
        epsilon = 0.01 * cv2.arcLength(pad_cnt, False)
        approx = cv2.approxPolyDP(pad_cnt, epsilon, False)
        approx_area = cv2.contourArea(approx)

        # judgement
        err_area = (hull_area - approx_area)/approx_area*100
        
        if err_area >= 11:
            #'detection result: BAD or confused'
            RES = 0
            MESSAGE = report('shapeFilter')
        else:
            #'detection result: GOOD'
            RES = 1
            MESSAGE = report('shapeFilter')
    except Exception as e:
        MESSAGE = handleExcept('shapeFilter', e)
    
    # if any exception was casted, RES would be 9
    return RES, MESSAGE 


def surfaceFilter(gray):
    RES = 9
    MESSAGE = ''
    try:
        # all of the contour & hierachy
        ##deblur then blur
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2,2))
        cl1 = clahe.apply(gray)
        bll = cv2.bilateralFilter(cl1,3,30,30)

        # threshold
        threshll = threshold_otsu(bll)
        retll,outputll = cv2.threshold(bll,threshll,255,cv2.THRESH_BINARY)

        # find contours and draw to pad
        cnts, hierachy = cv2.findContours(outputll.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        # contour area & other contour area within the pad contour
        numms = int(np.array(cnts).shape[0])
        empty_temp = np.zeros((numms,1))
        empty_temp_big = empty_temp.copy()

        for num in range(numms):
            #print cv2.contourArea(cnts[num])
            empty_temp[num]=cv2.contourArea(cnts[num])

        max_index = np.argmax(empty_temp)
        max_contour = empty_temp[max_index]

        for num in range(numms):
            if hierachy[0][num][3] == max_index:
                empty_temp_big[num] = 1

        contour_within = empty_temp[np.where(empty_temp_big == 1)[0]]
        if  contour_within.shape[0]==0:
            RES = 1
        if  contour_within.shape[0]>0:
            if max(contour_within)<10:
                RES = 1   
        if (sum(contour_within)/max_contour)>=0.4 or (max(contour_within)/max_contour)>=0.1:
            RES = -1
        else:
            RES =  0
        MESSAGE = report('surfaceFilter')
        
    except Exception as e:
        MESSAGE = handleExcept('surfaceFilter', e)
        
    # if any exception was casted, RES would be 9
    return RES, MESSAGE 


def update_shape_filter(color_img):
    RES = 9
    MESSAGE = ''
    
    #  Setting interval & range of treshold
    interval = -10
    rag = (xrange(200, 100, interval))
    
    # init
    sim_list = []
    slope_list = []
    first_cnt = None
    judgment_sim = 0
    judgment_slope = 0
    
    # diff gray & blur
    gray = diff_scale(color_img)
    gray = cv2.convertScaleAbs(gray)
    bl = cv2.GaussianBlur(gray, (3, 3), 5.0)
    
    # erode & dilate
    kernel = cv2.getStructuringElement(cv2.MORPH_ERODE,(5, 5))
    er1 = cv2.erode(bl, kernel, 1)
    di_kernel = cv2.getStructuringElement(cv2.MORPH_DILATE, (5, 5))
    di1 = cv2.dilate(er1, di_kernel, 1)
    
    # scaling threshold
    try:
        for index, i in enumerate(rag):
            ret, th = cv2.threshold(di1, i, 255, cv2.THRESH_BINARY_INV)
            window1 = np.zeros(img.shape, dtype=np.uint8)
            cnts,_ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
            try:
                approx = cv2.approxPolyDP(cnts[0], 2, True)
                cv2.drawContours(window1, [approx], 0, (0, 0, 255), 1)

                # get the first cnt
                if index == 0:
                    first_cnt = cnts[0]

                # calculating the simularity of shape between different treshold
                cnt = cnts[0]
                err_sim = cv2.matchShapes(first_cnt, cnt, cv2.cv.CV_CONTOURS_MATCH_I3, 0.0)
                sim_list.append(err_sim*100)
            except IndexError:
                sim_list.append(None)
                handleExcept
            except ZeroDivisionError:
                sim_list.append(100.0)
            
            # counting the times of simularity err>20
            if err_sim*100 > 20.0:
                judgment_sim += 1
        
        # calculating the slope of the shape simularity
        for i in xrange(0, len(sim_list)):
            try:
                slope = (sim_list[i+1]-sim_list[i])/-interval
            except IndexError as e:
                pass
            
            # counting the times of the slope>0.5 
            if slope > 0.5:
                judgment_slope += 1
            slope_list.append(slope)
        
        #print sim_list
        #print slope_list
        
        # final judgment
        if judgment_sim == 0 and judgment_slope == 0:
            RES = 1
            MESSAGE = report('update_shape_filter')
        elif judgment_sim >= 1 and judgment_slope >= 1:
            RES = -1
            MESSAGE = report('update_shape_filter')
        else:
            RES = 0
            MESSAGE = report('update_shape_filter')
            
    except Exception as e:
        MESSAGE = handleExcept('update_shape_filter', e)
    return RES, MESSAGE     

In [17]:
path = '\\\\192.168.1.192\\Vision\\crop\\bad\\S50036-sensofar_20x_image_white_coaxial.jpg'
img = getImg(path)
gray = grayImg(img)
res1, msg1 = shapeFilter(gray)
res2, msg2 = surfaceFilter(gray)
res3, msg3 = update_shape_filter(img)
print res1, msg1
print res2, msg2
print res3, msg3

0 shapeFilter : succeed
0 surfaceFilter : succeed
-1 update_shape_filter : succeed


In [15]:
# if input is not color image:
path = '\\\\192.168.1.192\\Vision\\crop\\bad\\S50036-sensofar_20x_image_white_coaxial.jpg'
img = getImg(path)
gray = grayImg(img)
res3, msg3 = update_shape_filter(gray)
print res3, msg3

RuntimeError: Input img is not a three-chennel array

In [2]:
import cv2

In [3]:
cv2.

['ADAPTIVE_SKIN_DETECTOR_MORPHING_METHOD_ERODE',
 'ADAPTIVE_SKIN_DETECTOR_MORPHING_METHOD_ERODE_DILATE',
 'ADAPTIVE_SKIN_DETECTOR_MORPHING_METHOD_ERODE_ERODE',
 'ADAPTIVE_SKIN_DETECTOR_MORPHING_METHOD_NONE',
 'ADAPTIVE_THRESH_GAUSSIAN_C',
 'ADAPTIVE_THRESH_MEAN_C',
 'ANN_MLP',
 'ANN_MLP_GAUSSIAN',
 'ANN_MLP_IDENTITY',
 'ANN_MLP_NO_INPUT_SCALE',
 'ANN_MLP_NO_OUTPUT_SCALE',
 'ANN_MLP_SIGMOID_SYM',
 'ANN_MLP_TRAIN_PARAMS_BACKPROP',
 'ANN_MLP_TRAIN_PARAMS_RPROP',
 'ANN_MLP_UPDATE_WEIGHTS',
 'Algorithm__create',
 'Algorithm_getList',
 'BFMatcher',
 'BOOST_DEFAULT',
 'BOOST_DISCRETE',
 'BOOST_GENTLE',
 'BOOST_GINI',
 'BOOST_LOGIT',
 'BOOST_MISCLASS',
 'BOOST_REAL',
 'BOOST_SQERR',
 'BORDER_CONSTANT',
 'BORDER_DEFAULT',
 'BORDER_ISOLATED',
 'BORDER_REFLECT',
 'BORDER_REFLECT101',
 'BORDER_REFLECT_101',
 'BORDER_REPLICATE',
 'BORDER_TRANSPARENT',
 'BORDER_WRAP',
 'BOWImgDescriptorExtractor',
 'BOWKMeansTrainer',
 'BRISK',
 'BackgroundSubtractorMOG',
 'BackgroundSubtractorMOG2',
 'Boost',
 'CAL